# Preparatory steps



In [ ]:
# access data from the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# alternative: upload files
# from google.colab import files
# uploaded = files.upload()

### Installing necessary packages

Install the necessary bert packages and Tensorflow 1.15.2

In [ ]:
!pip uninstall gast
!pip install gast==0.2.2
!pip install sentencepiece
!pip install bert-tensorflow==1.0.1
!pip install pandas==0.24

After restarting the runtime as suggested, run following:

In [ ]:
%tensorflow_version 1.x

In [ ]:
# for importing modules in .py files, insert the working directory
import sys
sys.path.insert(0,'/content/drive/My Drive/AIandLaw/CodeAndModel/')

# Create BERT features

With full training_data_hate_speech.csv.

This script is useful to create BERT features for a new model training set.
As this is a slow process it is separated from the training of the xgboost model.
The script only needs three inputs:
- data_input_path: Location of the data. Expects a csv file with a "Comment_text" and a "Hateful_or_not" column.
- data_output_path: Location where the BERT features will be saved.
    This file is used as input to "train_optimize_xgboost.py"
- bert_model_path: Location of the BERT model.

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf

from bert_functions_modified import *
from functions_modified import *

In [ ]:
# --------- User Input ----------
data_input_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/dataset/training_data_hate_speech.csv"
data_output_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/bert_features.csv"
bert_model_path = "/content/drive/My Drive/AIandLaw/CodeAndModel/bert_model_updated.h5" # if updated, change file name

In [ ]:
sess = tf.Session()
max_seq_length = 256
bert_model = build_model(max_seq_length, bert_path="https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")
initialize_vars(sess)
bert_model.load_weights(bert_model_path)

In [ ]:
X_train = pd.read_csv(data_input_path, sep=',')

In [ ]:
# Create datasets (Only take up to max_seq_length words for memory)
train_text = X_train['Comment_text'].tolist()
train_text = [' '.join(str(t).split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = X_train['Hateful_or_not'].tolist()

In [ ]:
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", sess)

In [ ]:
# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)

In [ ]:
# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)

In [ ]:
bert_train, bert_features = get_bert_features([train_input_ids, train_input_masks, train_segment_ids], bert_model)
X_train = X_train.reset_index().join(bert_train).set_index('index')

X_train.to_csv(data_output_path)